In [1]:
import warnings
warnings.simplefilter('ignore')

# imports

import pandas as pd
import numpy as np

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer

In [2]:
#read csv files in
df = pd.read_csv("NBA_data/NBA_data_all_80-19.csv")
df2020 = pd.read_csv("NBA_data/NBA_data_all_19-20.csv")

In [3]:
# creating new columns
df["playoffs_y_n"] = df["Team"].map(lambda x: 1 if "*" in x else 0)
df["Team"] = df["Team"].str.replace("*", "")
df["W%"] = df["W"]/df["G"]

df2020["playoffs_y_n"] = df2020["Team"].map(lambda x: 1 if "*" in x else 0)
df2020["Team"] = df2020["Team"].str.replace("*", "")
df2020["W%"] = df2020["W"]/df2020["G"]

In [4]:
# scale stats by year
scaler = QuantileTransformer(output_distribution='normal')
normalizer = Normalizer()

norm_scaled_dfs = []

# These columns will not be scaled/normalized.
stay_columns = ["RK", "Team", "Year", "Conf", "playoffs_y_n", "G", "W", "L"]

a = df["Year"].min()
b = df["Year"].max()
x = b - a + 1
for i in range (x):
    year = a+i
    df_year = df.loc[df["Year"] == year]
    stay_df = df_year[stay_columns].reset_index(drop=True)   
    change_df = df_year.drop(columns=stay_columns).reset_index(drop=True)
    
#   scale data
    scaled_df = pd.DataFrame(scaler.fit_transform(change_df), columns=change_df.columns)
#   normalize data
    normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df), columns=scaled_df.columns)    
    
    merged_df = stay_df.merge(change_df, left_index=True, right_index=True)
    merged_df = merged_df.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
    
    norm_scaled_dfs.append(merged_df)

final_df = pd.concat(norm_scaled_dfs, axis=0)
final_df = final_df.dropna()

# scale 2020 df
stay_df2020 = df2020[stay_columns].reset_index(drop=True)   
change_df2020 = df2020.drop(columns=stay_columns).reset_index(drop=True)

scaled_df2020 = pd.DataFrame(scaler.fit_transform(change_df2020), columns=change_df2020.columns)
normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df2020), columns=scaled_df2020.columns) 

merged_df2020 = stay_df2020.merge(change_df2020, left_index=True, right_index=True)
merged_df2020 = merged_df2020.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
final_df2020 = merged_df2020 

In [5]:
final_df2020.to_csv("NBA_data/preprocessed_2020_Wperc.csv")
final_df.to_csv("NBA_data/preprocessed_80-19_Wperc.csv")

In [6]:
final_df.columns

Index(['RK', 'Team', 'Year', 'Conf', 'playoffs_y_n', 'G', 'W', 'L', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W%',
       'MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')